# Exercise 6

(14 points)

Write a function that, given an invertible matrix $A$, returns a list of elementary matrices whose product is $A$.

Hint: Think of how we produced the inverse of a matrix as a product of elementary matrices by doing row operations, that transformed the matrix into the unit matrix. From the list of these row operations we can easily produce the required list of elementary matrices.
    
    
    

As a warmup, given a matrix $A$, you can produce a list consisting of elementary matrices and a single matrix in row echelon form, such that the product of the matrices in this list gives back A. For this you can freely modify the code below. If you get this far you get 7 points.

In [1]:
import numpy as np

In [2]:
# you can, but you don't need to use (parts of) this code
#    - feel free to write your solution from scratch, if you prefer that!
def row_echelon(given_matrix):
    """
    Returns a matrix in row echelon form obtained from the given_matrix.
    """
    # make a copy so that we don't change the original matrix
    mat = given_matrix.copy()
    nr_rows,nr_columns=mat.shape
    # The algorithm runs through all the columns
    start_row = 0
    for current_column in range(nr_columns):
        # every iteration ignores one more column, if we found a zero column, 
        # or one more row and column otherwise
        # in each iteration go down the current leftmost column:
        for r in range(start_row, nr_rows):
            # find the first row with a nonzero entry in first column
            if mat[r][current_column] == 0:
                # case 1: we have a zero entry, but are not yet at the bottom row. 
                # Go to the next iteration of the inner for loop
                # and consider the next entry in this column.
                if r!=nr_rows-1:
                    continue
                # case 2: we arrived at the last row so we have a zero column. 
                # Go back to the outer for loop and consider the next column    
                else:
                    break
            # if we arrived here we have found a non-zero entry in row r.
            # Then swap our starting row with the first row
            """The .copy() was missing here before, causing the attempted row switching to overwrite a row"""
            mat[start_row], mat[r] = mat[r].copy(), mat[start_row].copy()
            # now add multiples of the new first row to lower rows
            # such that lower entries of the current column become zero
            upper_left_entry = mat[start_row][current_column]
            for rr in range(start_row+1, nr_rows):
                entry_that_should_become_zero = mat[rr][current_column]
                scalar_multiple = -1 * entry_that_should_become_zero / upper_left_entry
                for cc in range(current_column, nr_columns):
                    mat[rr][cc] += mat[start_row][cc] * scalar_multiple
            # The current start row will not be changed anymore
            start_row += 1
    return mat

We tried to solve it on paper and verify the results with the code output, and it was working correctly but my logic needed modification to calculate the correct values of the elementary matrices in the below functions

In [14]:
A=np.array([[1.,2.,3.],[1.,1.,1.],[5.,2.,1.]])

E1=np.array([[1.,0.,0.],[-1.,1.,0.],[0.,0.,1.]])
E2=np.array([[1.,0.,0.],[0.,1.,0.],[-5.,0.,1.]])
E3=np.array([[1.,0.,0.],[0.,1.,0.],[0.,-8.,1.]])
R=np.array([[1.,2.,3.],[0.,-1.,-2.],[0.,0.,2.]])

result = E3@E2@E1@A
print(result)
print(row_echelon(A))

[[ 1.  2.  3.]
 [ 0. -1. -2.]
 [ 0.  0.  2.]]
[[ 1.  2.  3.]
 [ 0. -1. -2.]
 [ 0.  0.  2.]]


Our master pieces :)

In [ ]:
def elementary_matrices(given_matrix):
    E1 = np.eye(3)
    E2 = np.eye(3)
    E3 = np.eye(3)


    E1[1, 0] = -given_matrix[1, 0] / given_matrix[0, 0]
    E2[2, 0] = -given_matrix[2, 0] / given_matrix[0, 0]


    A1 = E1 @ given_matrix
    A2 = E2 @ A1

    E3[2, 1] = -A2[2, 1] / A2[1, 1]

    return [E1, E2, E3]

def inverse_elementary_matrices(E1, E2, E3):
    E1_inv = np.eye(3)
    E1_inv[1, 0] = -E1[1, 0]

    E2_inv = np.eye(3)
    E2_inv[2, 0] = -E2[2, 0]
    E2_inv[1, 1] = 1 / E2[1, 1]
    
    E3_inv = np.eye(3)
    E3_inv[2, 1] = -E3[2, 1]
    E3_inv[1, 1] = 1 / E3[1, 1]

    return [E1_inv, E2_inv, E3_inv]
    # ...a list of elementary matrices (given as numpy arrays) whose product is given_matrix

We started by trying to simply get the correct elementary matrices for the given matrix, then we adjusted the logic to fit any given 3x3 matrix not only with happy values like the given one :)

Our logic mainly depends on getting the elementary matrices first, verfiy by multiplying them descendingly with the original matrix it should return the echelon matrix

Then from the correct elementary matrices we calculated their inverse, verified by multiplying them ascendingly with the echelon matrix to get back the original matrix

In general for bigger matrices we can think about implementing the same logic, not with fixed steps (E1,E2,E3) but with for-loops depending on the size of the given matrix

In [50]:
A=np.array([[1.,2.,3.],[1.,1.,1.],[5.,2.,1.]])
echelon = row_echelon(A)
print(echelon)

B1, B2, B3 = elementary_matrices(A)
result = B3 @ B2 @ B1 @ A
print(result)

[[ 1.  2.  3.]
 [ 0. -1. -2.]
 [ 0.  0.  2.]]
[[ 1.  2.  3.]
 [ 0. -1. -2.]
 [ 0.  0.  2.]]


In [51]:
A=np.array([[1.,2.,3.],[1.,3.,1.],[5.,2.,1.]])
echelon = row_echelon(A)
print(echelon)

B1, B2, B3 = elementary_matrices(A)
result = B3 @ B2 @ B1 @ A
print(result)

[[  1.   2.   3.]
 [  0.   1.  -2.]
 [  0.   0. -30.]]
[[  1.   2.   3.]
 [  0.   1.  -2.]
 [  0.   0. -30.]]


In [55]:
A=np.array([[1.,2.,3.],[1.,3.,1.],[5.,2.,1.]])
echelon = row_echelon(A)
print(A)

B1, B2, B3 = elementary_matrices(A)
E1_inv, E2_inv, E3_inv = inverse_elementary_matrices(B1, B2, B3)
result = E1_inv @ E2_inv @ E3_inv @ echelon
print(result)

[[1. 2. 3.]
 [1. 3. 1.]
 [5. 2. 1.]]
[[1. 2. 3.]
 [1. 3. 1.]
 [5. 2. 1.]]


Your function should be such that the following code returns $\texttt{True}$ (also for other matrices than the concrete $A$ below, of course). Testing $\texttt{A==B}$ might not exactly return $\texttt{True}$ because of rounding errors or data type issues. So here is a test looking at whether the two matrices are close to each other.

(the test looks for the matrix distance in the $\ell_1$-norm on the vector space of matrices, in the terminology of the lecture. In practice the threshold can be taken to be much closer to zero than 0.01)

In [56]:
A=np.array([[1.,2.,3.],[1.,1.,1.],[5.,2.,1.]])
echelon = row_echelon(A)

B1, B2, B3 = elementary_matrices(A)
E1_inv, E2_inv, E3_inv = inverse_elementary_matrices(B1, B2, B3)
B = E1_inv @ E2_inv @ E3_inv @ echelon

def matrix_distance(A,B):
    sum_of_differences = 0
    for i in range(len(B)):
        for j in range(len(B[i])):
            sum_of_differences += abs(A[i,j]-B[i,j]) 
    return sum_of_differences

matrix_distance(A,B)<0.01

np.True_